# Policy gradient

## Алгоритм Reinforce
В некоторых задачах для нахождения удовлетворяющей стратегии необязательно изучать структуру всей среды. Например, в задаче поднятия кубика робототехнической рукой вместо точной аппроксимации $Q(s,a)$ достаточно знать, что выгоднее двигаться вправо, если кубик справа, и влево в ином случае. Алгоритм Reinforce (Monte Carlo policy gradient) - это алгоритм поиска стратегий, в котором параметры, задающие стохастическую стратегию, изменяются в соответствии с градиентом математического ожидания награды: 

$$J(\theta)=\mathbb E_{\tau\sim p_{\theta}(\tau)}(\sum_t\gamma^tr(s_t,a_t))$$

$$\theta\leftarrow\theta+\alpha\nabla_{\theta}J(\theta)$$

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
import gym

# Будем обучать агента, используя сначала простое окружение CartPole-v0,
# а затем усложним задачу взяв Acrobot-v1

# Создаем окружение 
# Здесь ваш код
env = gym.make("CartPole-v0")

observation_shape = env.observation_space.shape
n_actions = env.action_space.n
gamma = 0.95

print("Observation Space", env.observation_space)
print("Action Space", env.action_space)

Observation Space Box(4,)
Action Space Discrete(2)


### Задание 1. Определяем архитектуру сети
Стратегия задается весами нейронной сети и является стохастической, т.е. в состоянии $s$ она представляет себой некоторое распределение $\pi_\theta(s)$, поэтому на вход сети будет подаваться состояние $s$, а на выходе будут вероятности действий.

#### Задаем входы сети

In [0]:
# Задаем переменные, которые будут подаваться на вход нейронной сети

# Состояния
# Здесь ваш код
observations = tf.placeholder(tf.float32, [None, *observation_shape], name="observations")
# Совершенные действия
# Здесь ваш код
actions = tf.placeholder(tf.int32, [None, n_actions], name="actions")

# Вознаграждение
# Здесь ваш код
discounted_episode_rewards = tf.placeholder(tf.float32, [None], name="discounted_episode_rewards")

all_inputs = [observations, actions, discounted_episode_rewards]

#### Определяем граф вычислений

In [0]:
sess = tf.InteractiveSession()

# Задаем внутренние и выходной слои нейронной сети
# Здесь ваш код
nn1 = tf.contrib.layers.fully_connected(inputs=observations, num_outputs=50, activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.xavier_initializer())

nn2 = tf.contrib.layers.fully_connected(inputs=nn1, num_outputs=20, activation_fn=tf.nn.relu, weights_initializer=tf.contrib.layers.xavier_initializer())
nn3 = tf.contrib.layers.fully_connected(inputs=nn2, num_outputs=n_actions, activation_fn=None, weights_initializer=tf.contrib.layers.xavier_initializer())
  

probs_out = tf.nn.softmax(nn3) 
# Выход последнего слоя преобразуется в стохастическую стратегию, 
# поэтому количество нейронов должно быть равно n_actions

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    
    # Считаем дисконтированное вознаграждение "G = r + gamma*r' + gamma^2*r'' + ..."
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    # Нормализуем данные
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    
    return discounted_episode_rewards

### Задание 2. Функция потерь

Теперь определим функцию потерь (Crossentropy loss)

Градиент стратегии выглядит следующим образом $\nabla_{\theta}J_{\theta}\approx\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T\nabla_{\theta}log\pi_{\theta}(a_{i,t}|s_{i,t})R $. Чтобы автоматически вычислить градиент, необходимо задать граф, который имеет градиент такого же вида. Для этого используется "псевдо" функция потерь: $$\tilde{J}_{\theta}=\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^Tlog\pi_{\theta}(a_{i,t}|s_{i,t})R $$.

Подробнее http://rail.eecs.berkeley.edu/deeprlcourse-fa17/index.html#lecture-videos (6 sep)

In [0]:
# Применяем кросс энтропию к ПОСЛЕДНЕМУ слою сети tf.nn.softmax_cross_entropy_with_logits_v2
# Здесь ваш код
neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=nn3, labels=actions)

# Умножаем на дисконтированное вознаграждение и берем среднее, чтобы получить искомую функцию потерь
# Здесь ваш код
loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards)     

# Задаем метод оптимизации, который будем использовать (например adam с lr 0.01)
# Здесь ваш код
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(loss, global_step=tf.contrib.framework.get_global_step())
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please switch to tf.train.get_global_step


Градиент имеет большую дисперсию. Для ее уменьшения можно использовать следующую функцию потерь: $$\tilde{J}_{\theta}=\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^Tlog\pi_{\theta}(a_{i,t}|s_{i,t})(R-b) $$.

$b$ - может быть константой (это не сильно улучшит работу алгоритма), а может быть функцией от $s$, например $V(s)$. $V(s)$ может быть аппроксимированна другой нейронной сетью. Настройка может проходить по методу наименьших квадратов: необходимо задать функцию ошибок и можно добавить ее к loss'у с некоторым коэффициентом или минимизировать отдельно. Пример: https://github.com/yrlu/reinforcement_learning/blob/master/policy_gradient/reinforce_w_baseline.py

In [0]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]
max_episodes = 10000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for episode in range(max_episodes):
        
        episode_rewards_sum = 0
        state = env.reset()
        # Можно посмотреть как выглядит окружение, если мы не используем google colab
#         env.render()
           
        while True:
            
            # Получаем распределение вероятностей действий в соответствии со стохастической стратегией агента
            # Здесь ваш код
            probs = sess.run(probs_out, feed_dict = {
                observations : state.reshape([1, env.observation_space.shape[0]])})
            
            # Выбираем действие (согласно распределению)
            # Здесь ваш код
            action =  np.random.choice(range(probs.shape[1]), p=probs.ravel())
            
            # Применяем действие в среде 
            # Здесь ваш код
            new_state, reward, done, info = env.step(action)

            episode_states.append(state)
            action_ = np.zeros(n_actions)
            action_[action] = 1
            
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            if done:
                episode_rewards_sum = np.sum(episode_rewards)
                
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                
                mean_reward = np.divide(total_rewards, episode+1)
                
                
                maximumRewardRecorded = np.amax(allRewards)
                if episode % 50 == 0:
                    print("==========================================")
                    print("Episode: ", episode)
                    print("Reward: ", episode_rewards_sum)
                    print("Mean Reward", mean_reward)
                    print("Max reward so far: ", maximumRewardRecorded)
                
                # Считаем дисконтированное вознаграждение
                # Здесь ваш код
                discounted_rewards = discount_and_normalize_rewards(episode_rewards)
                                
                # Считаем значение функции потерь и настраиваем веса сети при помощи оптимизатора
                # Здесь ваш код
                loss_, _ = sess.run([loss, train_op], feed_dict={
                    observations: np.vstack(np.array(episode_states)),
                    actions: np.vstack(np.array(episode_actions)), 
                    discounted_episode_rewards: discounted_rewards
                })
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            state = new_state

Episode:  0
Reward:  14.0
Mean Reward 14.0
Max reward so far:  14.0
Episode:  50
Reward:  200.0
Mean Reward 109.17647058823529
Max reward so far:  200.0
Episode:  100
Reward:  35.0
Mean Reward 125.29702970297029
Max reward so far:  200.0
Episode:  150
Reward:  119.0
Mean Reward 140.71523178807948
Max reward so far:  200.0
Episode:  200
Reward:  200.0
Mean Reward 145.33830845771143
Max reward so far:  200.0
Episode:  250
Reward:  200.0
Mean Reward 156.22709163346613
Max reward so far:  200.0
Episode:  300
Reward:  200.0
Mean Reward 157.77408637873754
Max reward so far:  200.0
Episode:  350
Reward:  200.0
Mean Reward 163.7891737891738
Max reward so far:  200.0
Episode:  400
Reward:  200.0
Mean Reward 168.30423940149626
Max reward so far:  200.0
Episode:  450
Reward:  200.0
Mean Reward 169.549889135255
Max reward so far:  200.0
Episode:  500
Reward:  105.0
Mean Reward 167.1996007984032
Max reward so far:  200.0
Episode:  550
Reward:  200.0
Mean Reward 168.32486388384754
Max reward so far:

### Задание 3. Проводим эксперименты.

Постройте график получаемого суммарного вознаграждения от номера эпизода.

Сделайте тоже самое для задачи CartPole-v0. 

Усложните архитектуру сети. Улучшает ли это производительность?

### Дополнительные материалы

Пример алгоритма DDPG (https://arxiv.org/pdf/1509.02971.pdf) - алгоритм использующий градиент стратегий и позволяющий работать с непрерывным множеством действий:

https://pemami4911.github.io/blog/2016/08/21/ddpg-rl.html